In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import shutil
import sys
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
import tensorflow as tf
import bottleneck as bn
import keras
import math

In [3]:
import argparse
import pickle
import time
import numpy as np

In [4]:
cd /content/drive/MyDrive/Colab Notebooks/recommendation_system/deep_learning/session_based_rec/TAGNN-master/

/content/drive/MyDrive/Colab Notebooks/recommendation_system/deep_learning/session_based_rec/TAGNN-master


In [5]:
train_data = pickle.load(open('../datasets/' + 'diginetica' + '/train.txt', 'rb'))

In [6]:

def build_graph(train_data):
    graph = nx.DiGraph()
    for seq in train_data:
        for i in range(len(seq) - 1):
            if graph.get_edge_data(seq[i], seq[i + 1]) is None:
                weight = 1
            else:
                weight = graph.get_edge_data(seq[i], seq[i + 1])['weight'] + 1
            graph.add_edge(seq[i], seq[i + 1], weight=weight)
    for node in graph.nodes:
        sum = 0
        for j, i in graph.in_edges(node):
            sum += graph.get_edge_data(j, i)['weight']
        if sum != 0:
            for j, i in graph.in_edges(i):
                graph.add_edge(j, i, weight=graph.get_edge_data(j, i)['weight'] / sum)
    return graph


def data_masks(all_usr_pois, item_tail):
    us_lens = [len(upois) for upois in all_usr_pois]
    len_max = max(us_lens)
    us_pois = [upois + item_tail * (len_max - le) for upois, le in zip(all_usr_pois, us_lens)]
    us_msks = [[1] * le + [0] * (len_max - le) for le in us_lens]
    return us_pois, us_msks, len_max


def split_validation(train_set, valid_portion):
    train_set_x, train_set_y = train_set
    n_samples = len(train_set_x)
    sidx = np.arange(n_samples, dtype='int32')
    np.random.shuffle(sidx)
    n_train = int(np.round(n_samples * (1. - valid_portion)))
    valid_set_x = [train_set_x[s] for s in sidx[n_train:]]
    valid_set_y = [train_set_y[s] for s in sidx[n_train:]]
    train_set_x = [train_set_x[s] for s in sidx[:n_train]]
    train_set_y = [train_set_y[s] for s in sidx[:n_train]]

    return (train_set_x, train_set_y), (valid_set_x, valid_set_y)


class Data():
    def __init__(self, data, shuffle=False, graph=None):
        inputs = data[0]
        inputs, mask, len_max = data_masks(inputs, [0])
        self.inputs = np.asarray(inputs)
        self.mask = np.asarray(mask)
        self.len_max = len_max
        self.targets = np.asarray(data[1])
        self.length = len(inputs)
        self.shuffle = shuffle
        self.graph = graph

    def generate_batch(self, batch_size):
        if self.shuffle:
            shuffled_arg = np.arange(self.length)
            np.random.shuffle(shuffled_arg)
            self.inputs = self.inputs[shuffled_arg]
            self.mask = self.mask[shuffled_arg]
            self.targets = self.targets[shuffled_arg]
        n_batch = int(self.length / batch_size)
        if self.length % batch_size != 0:
            n_batch += 1
        slices = np.split(np.arange(n_batch * batch_size), n_batch)
        slices[-1] = slices[-1][:(self.length - batch_size * (n_batch - 1))]
        return slices

    def get_slice(self, i):
        inputs, mask, targets = self.inputs[i], self.mask[i], self.targets[i]
        items, n_node, A, alias_inputs = [], [], [], []
        for u_input in inputs:
            n_node.append(len(np.unique(u_input)))
        max_n_node = np.max(n_node)
        for u_input in inputs:
            node = np.unique(u_input)
            items.append(node.tolist() + (max_n_node - len(node)) * [0])
            u_A = np.zeros((max_n_node, max_n_node))
            for i in np.arange(len(u_input) - 1):
                if u_input[i + 1] == 0:
                    break
                u = np.where(node == u_input[i])[0][0]
                v = np.where(node == u_input[i + 1])[0][0]
                u_A[u][v] = 1
            u_sum_in = np.sum(u_A, 0)
            u_sum_in[np.where(u_sum_in == 0)] = 1
            u_A_in = np.divide(u_A, u_sum_in)
            u_sum_out = np.sum(u_A, 1)
            u_sum_out[np.where(u_sum_out == 0)] = 1
            u_A_out = np.divide(u_A.transpose(), u_sum_out)
            u_A = np.concatenate([u_A_in, u_A_out]).transpose()
            A.append(u_A)
            alias_inputs.append([np.where(node == i)[0][0] for i in u_input])
        return alias_inputs, A, items, mask, targets


In [7]:
train_data, valid_data = split_validation(train_data, 0.1)
test_data = valid_data

In [8]:
train_data1 = Data(train_data, shuffle=False)
test_data1 = Data(test_data, shuffle=False)

In [9]:
class opt:
  def __init__(self,step,hidden_size,batch_size,nonhybrid=None):
    self.step=step
    self.hiddenSize = hidden_size
    #self.n_node = n_node
    self.batchSize = batch_size
    self.nonhybrid = nonhybrid
    self.lr = 0.001
    self.l2 = 1e-5
    self.lr_dc_step = 3
    self.lr_dc = 0.1
    self.epoch = 30
    self.patience = 10
    

In [10]:
oopt=opt(step = 1,hidden_size=100,batch_size=100)

In [11]:
n_node = 43098

In [12]:
class GNN(tf.keras.layers.Layer):
    def __init__(self, hidden_size, step=1):
        super(GNN, self).__init__()
        self.step = step
        self.hidden_size = hidden_size
        self.input_size = hidden_size * 2
        self.gate_size = 3 * hidden_size
        # self.w_ih = Parameter(torch.Tensor(self.gate_size, self.input_size))
        self.stdv = 1/math.sqrt(self.hidden_size)
        self.w_ih = self.add_weight(shape=(self.input_size,self.gate_size),
                               initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        '''
        AttributeError: module 'keras.api._v2.keras.initializers' has no attribute 'Uniform'
        add_weight doesn't have the attribute named 'kernel_initializer'
        '''
        # self.w_hh = Parameter(torch.Tensor(self.gate_size, self.hidden_size))
        self.w_hh = self.add_weight(shape=(self.hidden_size,self.gate_size),
                               initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        # self.b_ih = Parameter(torch.Tensor(self.gate_size))
        self.b_ih = self.add_weight(shape=(self.gate_size,),
                               initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        
        # self.b_hh = Parameter(torch.Tensor(self.gate_size))
        self.b_hh = self.add_weight(shape=(self.gate_size,),
                               initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
                
        # self.b_iah = Parameter(torch.Tensor(self.hidden_size))
        self.b_iah = self.add_weight(shape=(self.hidden_size,),
                               initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        # self.b_oah = Parameter(torch.Tensor(self.hidden_size))
        self.b_oah = self.add_weight(shape=(self.hidden_size,),
                               initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))

        # self.linear_edge_in = nn.Linear(self.hidden_size, self.hidden_size, bias=True)
        self.linear_edge_in = tf.keras.layers.Dense(self.hidden_size, 
                                                    use_bias=True,
                               kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                               bias_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        '''
        TypeError: ('Unknown keyword argument:', 'bias') - > use_bias=True in Dense
        '''
        # self.linear_edge_out = nn.Linear(self.hidden_size, self.hidden_size, bias=True)
        self.linear_edge_out = tf.keras.layers.Dense(self.hidden_size,
                                    use_bias=True ,
                               kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                               bias_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        # self.linear_edge_f = nn.Linear(self.hidden_size, self.hidden_size, bias=True)
        self.linear_edge_f = tf.keras.layers.Dense(self.hidden_size,
                                    use_bias=True ,
                               kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                               bias_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))

    def Cell(self, A, hidden):
        input_in = tf.matmul(A[:, :, :A.shape[1]], self.linear_edge_in(hidden)) + self.b_iah
        input_out = tf.matmul(A[:, :, A.shape[1]: 2 * A.shape[1]],self.linear_edge_out(hidden)) + self.b_oah
        inputs = tf.concat([input_in, input_out], 2) 
        gi = tf.matmul(inputs, self.w_ih )+ self.b_ih # Dense layer와 같은 역할함.
        # gate_size가 앞서서 3* hidden_size로 정의되어 있었고 아래 chunk에서 split하는 것 같다.
        gh = tf.matmul(hidden, self.w_hh) + self.b_hh
        i_r, i_i, i_n = tf.split(gi,3,axis = 2)
        h_r, h_i, h_n = tf.split(gh,3,axis = 2)
        resetgate = tf.nn.sigmoid(i_r + h_r)
        inputgate = tf.nn.sigmoid(i_i + h_i)
        newgate = tf.nn.tanh(i_n + resetgate * h_n)
        hy = newgate + inputgate * (hidden - newgate)
        return hy

    def call(self, A, hidden):
        for i in range(self.step):
            hidden = self.Cell(A, hidden)
        return hidden

In [13]:

class TAGNN(tf.keras.models.Model):
    def __init__(self, opt, n_node):
        super(TAGNN, self).__init__()
        self.hidden_size = opt.hiddenSize
        self.n_node = n_node
        self.batch_size = opt.batchSize
        self.nonhybrid = opt.nonhybrid
        # self.embedding = nn.Embedding(self.n_node, self.hidden_size)
        self.embedding = tf.keras.layers.Embedding(input_dim = self.n_node,output_dim=self.hidden_size)
        self.stdv = 1/math.sqrt(self.hidden_size)
        
        self.gnn = GNN(self.hidden_size, step=opt.step)

        # self.linear_one = nn.Linear(self.hidden_size, self.hidden_size, bias=True)
        self.linear_one = tf.keras.layers.Dense(self.hidden_size, use_bias=True,
                                                kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                                                bias_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        # self.linear_two = nn.Linear(self.hidden_size, self.hidden_size, bias=True)
        self.linear_two = tf.keras.layers.Dense(self.hidden_size, use_bias=True,
                                                kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                                                bias_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        # self.linear_three = nn.Linear(self.hidden_size, 1, bias=False)
        self.linear_three = tf.keras.layers.Dense(1 , use_bias=False,
                                                  kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                                                  )
        # self.linear_transform = nn.Linear(self.hidden_size * 2, self.hidden_size, bias=True)
        
        self.linear_transform = tf.keras.layers.Dense(self.hidden_size, use_bias=True,
                                                      kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                                                      bias_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))

        # self.linear_t = nn.Linear(self.hidden_size, self.hidden_size, bias=False)  #target attention
        self.linear_t = tf.keras.layers.Dense( self.hidden_size, use_bias=False,
                                              kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                                              )
    def call(self, inputs, A, mask,alias_inputs):
        if len(self.embedding.weights) != 0:
          norm = tf.norm(self.embedding.weights[0],axis=1) # 왜 여기도 하는지는 모르겠음.
          self.embedding.weights[0]=self.embedding.weights[0] / tf.reshape(norm,[-1,1])
        hidden = self.embedding(inputs) 
        # inputs = items -> batch_size, len_seq_in_batch


        hidden = self.gnn(A, hidden)
        get = lambda j: tf.gather(hidden[j],alias_inputs[j])
        hidden = tf.stack([get(j) for j in range(len(alias_inputs))])

        seq_shape = hidden.shape # 여기 해보아야 알듯 리스트로 차원반환하는 것 같은데
        # seq_hidden = seq_hidden.view(-1, model.hidden_size) # tf.reshape
        hidden = tf.reshape(hidden,[-1,self.hidden_size])
        # norms = torch.norm(seq_hidden, p=2, dim=1)  # tf.norm
        norms = tf.norm(hidden,axis =1)
        # seq_hidden = seq_hidden.div(norms.unsqueeze(-1).expand_as(seq_hidden)) # unsqueeze에 대해 알아보기
        hidden = hidden / tf.reshape(norms,[-1,1])
        # expand_as는 broadcasting으로 해결될 수 있는지 확인해보기.
        # seq_hidden = seq_hidden.view(seq_shape) #  
        
        hidden = tf.reshape(hidden,seq_shape)

        #print(self.embedding.weights)
        ht = tf.gather_nd(hidden ,indices = tf.stack([tf.range(mask.shape[0],dtype='int64') , tf.cast(tf.math.reduce_sum(mask,1),'int64') -1],axis=1))
        # tf.reshape을 이용해야함.
        q1 = tf.reshape(self.linear_one(ht) ,[ht.shape[0], 1, ht.shape[1]])  # batch_size x 1 x latent_size
        q2 = self.linear_two(hidden)  # batch_size x seq_length x latent_size
        alpha = self.linear_three(tf.nn.sigmoid(q1 + q2))  # (b,s,1)
        
        alpha = tf.nn.softmax(alpha, 1) # B,S,1
        
        
        a = tf.math.reduce_sum(alpha * hidden * tf.cast(tf.reshape(mask, [mask.shape[0], -1, 1]),dtype = 'float32'), 1)  # (b,d)
        if not self.nonhybrid: # 여기 적용되는지 판단할 것
        
            a = self.linear_transform(tf.concat([a, ht], 1)) 
        
        norm = tf.norm(self.embedding.weights[0],axis=1)
        self.embedding.weights[0]=self.embedding.weights[0] / tf.reshape(norm,[-1,1])

        b = self.embedding.weights[0][1:] # 이 차원이 맞긴 함.

        hidden = hidden * tf.cast(tf.reshape(mask,[mask.shape[0], -1, 1]),'float32')  # batch_size x seq_length x latent_size
        

        qt = self.linear_t(hidden)  # batch_size x seq_length x latent_size

        beta = tf.nn.softmax(b @ tf.transpose(qt,[0,2,1]), -1)  # batch_size x n_nodes x seq_length
        target = beta @ hidden  # batch_size x n_nodes x latent_size
        a = tf.reshape(a,[ht.shape[0], 1, ht.shape[1]])  # b,1,d
        a = a + target  # b,n,d # 이거 concat인가 ? 

        scores = tf.math.reduce_sum(a * b, -1)  # b,n
        scores=tf.nn.softmax(scores,-1)
        return scores

# NISER 논문에서 나온 것처럼 처음 모델이 gnn이전에 embedding layer의 weights을 normalize하는 것을 call 안에서 짜려고 했는데, 처음 embedding layer가 호출 되기 이전까지 weight이 존재하지 않아서 다른 방식으로 짜야할 것 같음.

- build를 따로 둔다거나?
- Input을 사용해서 정의하는 것 ?
sequential이나
funtional api에서 하는 것처럼 일반적인 keras
built in을 사용하면 된다 ?


- 일단 미봉책으로 weight이 길이가 0인 경우 pass 한 다음 이후 embedding layer의 weights이 생겨난 후부터 normalize를 적용하도록 했음.



```python
import numpy as np

inputs = keras.Input(shape=(3,))
outputs = ActivityRegularizationLayer()(inputs)
model = keras.Model(inputs, outputs)

# If there is a loss passed in `compile`, the regularization
# losses get added to it
model.compile(optimizer="adam", loss="mse")
model.fit(np.random.random((2, 3)), np.random.random((2, 3)))

# It's also possible not to pass any loss in `compile`,
# since the model already has a loss to minimize, via the `add_loss`
# call during the forward pass!
model.compile(optimizer="adam")
model.fit(np.random.random((2, 3)), np.random.random((2, 3)))
```

In [14]:
model=TAGNN(oopt,n_node)

loss_ftn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate = oopt.lr)


@tf.function
def train_ftn(item_input,A_input,mask_input,alias_inputs_,targets_input):
    with tf.GradientTape() as tape :
      scores = model(item_input, A_input, mask_input,alias_inputs_,training =True)
      
      ll = loss_ftn(targets_input-1,scores)

    
    
    gradients =tape.gradient(ll,model.trainable_weights)

    optimizer.apply_gradients(zip(gradients,model.trainable_weights))
    return ll

In [15]:
@tf.function
def test_ftn(item_input,A_input,mask_input,alias_inputs_,targets_input):
    scores = model(item_input, A_input, mask_input,alias_inputs_,training =False)
    scores = tf.math.top_k(scores,k=20).indices
    scores = tf.cast(scores,tf.int64)
    targets_input = tf.cast(targets_input,tf.int64)

    #print(scores == targets_input[:,tf.newaxis]-1)
    result=tf.cast(scores == (targets_input[:,tf.newaxis]-1),tf.float32)

    result_hit = tf.math.reduce_sum(result,axis =1)
    
    result_hit = tf.cast(result_hit != 0,tf.float32) # hit

    index=tf.cast(tf.range(result.shape[-1])+1,tf.float32)


    result_mrr = tf.reduce_sum(result/tf.reshape(index,[-1,index.shape[0]]), axis =-1)


    return result_hit, result_mrr

In [16]:
for epoch in range(oopt.epoch):
    print('-------------------------------------------------------')
    print('epoch: ', epoch)

    slices = train_data1.generate_batch(model.batch_size)
    loss = 0
    for i, j in zip(slices, np.arange(len(slices))):
        
        alias_inputs, A, items, mask, targets = train_data1.get_slice(i)
        
        alias_inputs = tf.convert_to_tensor(alias_inputs)
        items = tf.convert_to_tensor(items)
        
        A = tf.convert_to_tensor(A)
        mask = tf.convert_to_tensor(mask)
        targets = tf.convert_to_tensor(targets)

        l = train_ftn(items,A,mask,alias_inputs,targets)
        
        loss += l 
        
        if j % int(100) == 0:
            print('[%d/%d] Loss: %.4f' % (j, len(slices), l))
        
    print('\tLoss:\t%.3f' % loss)
    hit, mrr = tf.constant([],dtype= tf.float32), tf.constant([],dtype= tf.float32)
    hhit=[]
    mmrr=[]
    slices = test_data1.generate_batch(model.batch_size)
    for i in slices:
        alias_inputs, A, items, mask, targets = test_data1.get_slice(i)
        
        alias_inputs = tf.convert_to_tensor(alias_inputs)
        items = tf.convert_to_tensor(items)
        
        A = tf.convert_to_tensor(A)
        mask = tf.convert_to_tensor(mask)
        targets = tf.convert_to_tensor(targets)

        h,m=test_ftn(items,A,mask,alias_inputs,targets)

        hit = tf.concat([hit,h],0)
        mrr = tf.concat([mrr,m],0)
        '''
        scores = model(items, A, mask,alias_inputs,training =False)
        scores = tf.math.top_k(scores,k=20).indices
        scores = scores.numpy()
        
            # sub_scores를 tensor에서 numpy로 바꾸고 아래는 그대로 실행하면 됨.
        for score, target, mask in zip(scores, targets, test_data1.mask):
            hhit.append(np.isin(target - 1, score)) # 현재 target은 s_{n+1}인 item이므로, 
                # score 즉, 위에서 추출한 index가 일치하면 hit이라는 list에 append 해준다.

                # score vector에서 target-1과 일치하는 경우를 찾고, 없는 경우 0을 mrr이라는 list에 기록하고,
            if len(np.where(score == target - 1)[0]) == 0:
                mmrr.append(0)
            else:
                # 있는 경우에는 (1/그 위치의 index +1)을 기록한다.
                mmrr.append(1 / (np.where(score == target - 1)[0][0] + 1))
        '''

    hit = np.mean(hit) * 100
    mrr = np.mean(mrr) * 100


    print(f"Recall@20 : {hit} , MRR@20 {mrr}")


-------------------------------------------------------
epoch:  0
[0/6476] Loss: 10.6664
[100/6476] Loss: 10.1675
[200/6476] Loss: 10.1818
[300/6476] Loss: 10.0300
[400/6476] Loss: 10.0923
[500/6476] Loss: 9.8792
[600/6476] Loss: 9.7917
[700/6476] Loss: 9.7979
[800/6476] Loss: 9.8881
[900/6476] Loss: 9.5757
[1000/6476] Loss: 9.8208
[1100/6476] Loss: 9.4335
[1200/6476] Loss: 9.2992
[1300/6476] Loss: 9.1560
[1400/6476] Loss: 9.2365
[1500/6476] Loss: 9.2054
[1600/6476] Loss: 8.8376
[1700/6476] Loss: 8.6460
[1800/6476] Loss: 8.5196
[1900/6476] Loss: 8.6320
[2000/6476] Loss: 8.6246
[2100/6476] Loss: 8.8561
[2200/6476] Loss: 8.5605
[2300/6476] Loss: 8.3657
[2400/6476] Loss: 7.9165
[2500/6476] Loss: 7.7918
[2600/6476] Loss: 7.8885
[2700/6476] Loss: 7.7251
[2800/6476] Loss: 7.5949
[2900/6476] Loss: 7.7721
[3000/6476] Loss: 7.7185
[3100/6476] Loss: 7.5099
[3200/6476] Loss: 7.4039
[3300/6476] Loss: 6.6640
[3400/6476] Loss: 7.0281
[3500/6476] Loss: 7.0076
[3600/6476] Loss: 6.9978
[3700/6476] Loss

KeyboardInterrupt: ignored

```
-------------------------------------------------------
epoch:  0
[0/6476] Loss: 10.6664
WARNING:tensorflow:5 out of the last 7 calls to <function train_ftn at 0x7fe1a010df80> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
WARNING:tensorflow:6 out of the last 9 calls to <function train_ftn at 0x7fe1a010df80> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
[100/6476] Loss: 10.1675
[200/6476] Loss: 10.1818
[300/6476] Loss: 10.0300
[400/6476] Loss: 10.0923
[500/6476] Loss: 9.8792
[600/6476] Loss: 9.7917
[700/6476] Loss: 9.7979
[800/6476] Loss: 9.8881
[900/6476] Loss: 9.5757
[1000/6476] Loss: 9.8208
[1100/6476] Loss: 9.4335
[1200/6476] Loss: 9.2992
[1300/6476] Loss: 9.1560
[1400/6476] Loss: 9.2365
[1500/6476] Loss: 9.2054
[1600/6476] Loss: 8.8376
[1700/6476] Loss: 8.6460
[1800/6476] Loss: 8.5196
[1900/6476] Loss: 8.6320
[2000/6476] Loss: 8.6246
[2100/6476] Loss: 8.8561
[2200/6476] Loss: 8.5605
[2300/6476] Loss: 8.3657
[2400/6476] Loss: 7.9165
[2500/6476] Loss: 7.7918
[2600/6476] Loss: 7.8885
[2700/6476] Loss: 7.7251
[2800/6476] Loss: 7.5949
[2900/6476] Loss: 7.7721
[3000/6476] Loss: 7.7185
[3100/6476] Loss: 7.5099
[3200/6476] Loss: 7.4039
[3300/6476] Loss: 6.6640
[3400/6476] Loss: 7.0281
[3500/6476] Loss: 7.0076
[3600/6476] Loss: 6.9978
[3700/6476] Loss: 6.8417
[3800/6476] Loss: 7.0825
[3900/6476] Loss: 7.1446
[4000/6476] Loss: 6.7950
[4100/6476] Loss: 6.8559
[4200/6476] Loss: 6.5710
[4300/6476] Loss: 6.1162
[4400/6476] Loss: 6.2964
[4500/6476] Loss: 6.6604
[4600/6476] Loss: 6.5483
[4700/6476] Loss: 6.3855
[4800/6476] Loss: 6.4676
[4900/6476] Loss: 6.8798
[5000/6476] Loss: 6.1916
[5100/6476] Loss: 6.5379
[5200/6476] Loss: 6.4180
[5300/6476] Loss: 6.0562
[5400/6476] Loss: 6.0324
[5500/6476] Loss: 5.7630
[5600/6476] Loss: 6.4351
[5700/6476] Loss: 5.8666
[5800/6476] Loss: 5.8163
[5900/6476] Loss: 6.5662
[6000/6476] Loss: 6.4659
[6100/6476] Loss: 5.8813
[6200/6476] Loss: 5.6327
[6300/6476] Loss: 5.7520
[6400/6476] Loss: 6.0481
	Loss:	49601.637
WARNING:tensorflow:5 out of the last 5 calls to <function test_ftn at 0x7fe1a00fbf80> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
WARNING:tensorflow:6 out of the last 7 calls to <function test_ftn at 0x7fe1a00fbf80> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
Recall@20 : 45.94910144805908 , MRR@20 15.47863781452179
-------------------------------------------------------
epoch:  1
[0/6476] Loss: 5.6468
[100/6476] Loss: 5.0910
[200/6476] Loss: 5.9533
[300/6476] Loss: 5.9028
[400/6476] Loss: 5.6767
[500/6476] Loss: 5.4639
[600/6476] Loss: 5.0921
[700/6476] Loss: 5.3682
[800/6476] Loss: 5.6095
[900/6476] Loss: 5.3022
[1000/6476] Loss: 5.4248
[1100/6476] Loss: 5.7704
[1200/6476] Loss: 5.2641
[1300/6476] Loss: 5.3862
[1400/6476] Loss: 5.4384
[1500/6476] Loss: 5.7164
[1600/6476] Loss: 5.3722
[1700/6476] Loss: 5.3273
[1800/6476] Loss: 5.5630
[1900/6476] Loss: 5.4944
[2000/6476] Loss: 5.9133
[2100/6476] Loss: 6.0222
[2200/6476] Loss: 5.5101
[2300/6476] Loss: 5.1625
[2400/6476] Loss: 5.4395
[2500/6476] Loss: 5.6007
[2600/6476] Loss: 5.4568
[2700/6476] Loss: 5.5793
[2800/6476] Loss: 5.2190
[2900/6476] Loss: 5.6015
[3000/6476] Loss: 6.0796
[3100/6476] Loss: 5.4805
[3200/6476] Loss: 5.7106
[3300/6476] Loss: 5.3003
[3400/6476] Loss: 5.3963
[3500/6476] Loss: 5.4450
[3600/6476] Loss: 5.4073
[3700/6476] Loss: 5.0991
[3800/6476] Loss: 5.5803
[3900/6476] Loss: 5.5295
[4000/6476] Loss: 5.2556
[4100/6476] Loss: 5.6052
[4200/6476] Loss: 5.4135
[4300/6476] Loss: 4.9751
[4400/6476] Loss: 5.1236
[4500/6476] Loss: 5.3566
[4600/6476] Loss: 5.4132
[4700/6476] Loss: 5.2663
[4800/6476] Loss: 5.2510
[4900/6476] Loss: 5.8611
[5000/6476] Loss: 5.1684
[5100/6476] Loss: 5.4179
[5200/6476] Loss: 5.2798
[5300/6476] Loss: 5.0541
[5400/6476] Loss: 5.0694
[5500/6476] Loss: 4.8808
[5600/6476] Loss: 5.2828
[5700/6476] Loss: 5.0148
[5800/6476] Loss: 4.9705
[5900/6476] Loss: 5.7491
[6000/6476] Loss: 5.6240
[6100/6476] Loss: 4.9554
[6200/6476] Loss: 4.9747
[6300/6476] Loss: 4.9222
[6400/6476] Loss: 5.1977
	Loss:	34934.660
Recall@20 : 51.96046829223633 , MRR@20 17.87794530391693
-------------------------------------------------------
epoch:  2
[0/6476] Loss: 4.9094
[100/6476] Loss: 4.4615
[200/6476] Loss: 5.1263
[300/6476] Loss: 5.0067
[400/6476] Loss: 5.0321
[500/6476] Loss: 4.7027
[600/6476] Loss: 4.5817
[700/6476] Loss: 4.7306
[800/6476] Loss: 4.9472
[900/6476] Loss: 4.8113
[1000/6476] Loss: 4.7923
[1100/6476] Loss: 5.2113
[1200/6476] Loss: 4.6482
[1300/6476] Loss: 4.7889
[1400/6476] Loss: 4.9753
[1500/6476] Loss: 5.0257
[1600/6476] Loss: 4.8502
[1700/6476] Loss: 4.7791
[1800/6476] Loss: 5.0209
[1900/6476] Loss: 4.7813
[2000/6476] Loss: 5.4252
[2100/6476] Loss: 5.5141
[2200/6476] Loss: 5.1028
[2300/6476] Loss: 4.6139
[2400/6476] Loss: 4.9963
[2500/6476] Loss: 5.2182
[2600/6476] Loss: 5.0153
[2700/6476] Loss: 5.2077
[2800/6476] Loss: 4.7296
[2900/6476] Loss: 5.1179
[3000/6476] Loss: 5.6129
[3100/6476] Loss: 4.9592
[3200/6476] Loss: 5.2777
[3300/6476] Loss: 4.8888
[3400/6476] Loss: 5.0433
[3500/6476] Loss: 4.9793
[3600/6476] Loss: 5.0519
[3700/6476] Loss: 4.6624
[3800/6476] Loss: 5.1424
[3900/6476] Loss: 5.0464
[4000/6476] Loss: 4.8013
[4100/6476] Loss: 5.1636
[4200/6476] Loss: 5.1060
[4300/6476] Loss: 4.5623
[4400/6476] Loss: 4.7473
[4500/6476] Loss: 5.0074
[4600/6476] Loss: 5.0303
[4700/6476] Loss: 4.9432
[4800/6476] Loss: 4.8376
[4900/6476] Loss: 5.4944
[5000/6476] Loss: 4.9044
[5100/6476] Loss: 5.0763
[5200/6476] Loss: 4.9301
[5300/6476] Loss: 4.6917
[5400/6476] Loss: 4.6698
[5500/6476] Loss: 4.5776
[5600/6476] Loss: 4.9169
[5700/6476] Loss: 4.7236
[5800/6476] Loss: 4.6718
[5900/6476] Loss: 5.4406
[6000/6476] Loss: 5.3456
[6100/6476] Loss: 4.6262
[6200/6476] Loss: 4.7729
[6300/6476] Loss: 4.6274
[6400/6476] Loss: 4.8585
	Loss:	31913.193
Recall@20 : 52.926456928253174 , MRR@20 18.35150569677353
-------------------------------------------------------
epoch:  3
[0/6476] Loss: 4.6953
[100/6476] Loss: 4.2113
[200/6476] Loss: 4.7006
[300/6476] Loss: 4.5875
[400/6476] Loss: 4.7244
[500/6476] Loss: 4.3312
[600/6476] Loss: 4.2614
[700/6476] Loss: 4.4524
[800/6476] Loss: 4.6209
[900/6476] Loss: 4.5871
[1000/6476] Loss: 4.4499
[1100/6476] Loss: 4.9245
[1200/6476] Loss: 4.3156
[1300/6476] Loss: 4.4614
[1400/6476] Loss: 4.7297
[1500/6476] Loss: 4.6495
[1600/6476] Loss: 4.6004
[1700/6476] Loss: 4.4788
[1800/6476] Loss: 4.6852
[1900/6476] Loss: 4.3986
[2000/6476] Loss: 5.0403
[2100/6476] Loss: 5.2296
[2200/6476] Loss: 4.8140
[2300/6476] Loss: 4.2305
[2400/6476] Loss: 4.7098
[2500/6476] Loss: 4.9589
[2600/6476] Loss: 4.7709
[2700/6476] Loss: 4.9002
[2800/6476] Loss: 4.4520
[2900/6476] Loss: 4.8307
[3000/6476] Loss: 5.2320
[3100/6476] Loss: 4.6384
[3200/6476] Loss: 5.0064
[3300/6476] Loss: 4.5600
[3400/6476] Loss: 4.7768
[3500/6476] Loss: 4.6627
[3600/6476] Loss: 4.8103
[3700/6476] Loss: 4.3957
[3800/6476] Loss: 4.8067
[3900/6476] Loss: 4.6955
[4000/6476] Loss: 4.4634
[4100/6476] Loss: 4.8666
[4200/6476] Loss: 4.8919
[4300/6476] Loss: 4.2786
[4400/6476] Loss: 4.4218
[4500/6476] Loss: 4.7916
[4600/6476] Loss: 4.7132
[4700/6476] Loss: 4.6339
[4800/6476] Loss: 4.5503
[4900/6476] Loss: 5.2514
[5000/6476] Loss: 4.7313
[5100/6476] Loss: 4.7919
[5200/6476] Loss: 4.6330
[5300/6476] Loss: 4.4723
[5400/6476] Loss: 4.3281
[5500/6476] Loss: 4.2606
[5600/6476] Loss: 4.6261
[5700/6476] Loss: 4.4649
[5800/6476] Loss: 4.3761
[5900/6476] Loss: 5.1859
[6000/6476] Loss: 5.1186
[6100/6476] Loss: 4.3204
[6200/6476] Loss: 4.5687
[6300/6476] Loss: 4.3446
[6400/6476] Loss: 4.5769
	Loss:	30024.197
Recall@20 : 52.31490135192871 , MRR@20 18.216778337955475
-------------------------------------------------------
epoch:  4
[0/6476] Loss: 4.5181
[100/6476] Loss: 3.9799
[200/6476] Loss: 4.3989
[300/6476] Loss: 4.2947
[400/6476] Loss: 4.4568
[500/6476] Loss: 4.0308
[600/6476] Loss: 4.0005
[700/6476] Loss: 4.2364
[800/6476] Loss: 4.3494
[900/6476] Loss: 4.3595
[1000/6476] Loss: 4.1301
[1100/6476] Loss: 4.7032
[1200/6476] Loss: 4.0801
[1300/6476] Loss: 4.1875
[1400/6476] Loss: 4.4986
[1500/6476] Loss: 4.3488
[1600/6476] Loss: 4.4005
[1700/6476] Loss: 4.2443
[1800/6476] Loss: 4.3827
[1900/6476] Loss: 4.1194
[2000/6476] Loss: 4.6644
[2100/6476] Loss: 4.9115
[2200/6476] Loss: 4.5402
[2300/6476] Loss: 3.9398
[2400/6476] Loss: 4.5055
[2500/6476] Loss: 4.6826
[2600/6476] Loss: 4.5615
[2700/6476] Loss: 4.6045
[2800/6476] Loss: 4.1917
[2900/6476] Loss: 4.5743
[3000/6476] Loss: 4.8736
[3100/6476] Loss: 4.3059
[3200/6476] Loss: 4.7784
[3300/6476] Loss: 4.2755
[3400/6476] Loss: 4.4780
[3500/6476] Loss: 4.3866
[3600/6476] Loss: 4.6010
[3700/6476] Loss: 4.1080
[3800/6476] Loss: 4.4202
[3900/6476] Loss: 4.4012
[4000/6476] Loss: 4.1215
[4100/6476] Loss: 4.5975
[4200/6476] Loss: 4.6336
[4300/6476] Loss: 4.0115
[4400/6476] Loss: 4.0906
[4500/6476] Loss: 4.5275
[4600/6476] Loss: 4.3579
[4700/6476] Loss: 4.3490
[4800/6476] Loss: 4.2836
[4900/6476] Loss: 4.9854
[5000/6476] Loss: 4.5176
[5100/6476] Loss: 4.5023
[5200/6476] Loss: 4.2968
[5300/6476] Loss: 4.2409
[5400/6476] Loss: 4.0096
[5500/6476] Loss: 4.0030
[5600/6476] Loss: 4.3249
[5700/6476] Loss: 4.2413
[5800/6476] Loss: 4.0904
[5900/6476] Loss: 4.8782
[6000/6476] Loss: 4.8803
[6100/6476] Loss: 4.0836
[6200/6476] Loss: 4.3519
[6300/6476] Loss: 4.0566
[6400/6476] Loss: 4.2498
	Loss:	28244.328
Recall@20 : 50.71650147438049 , MRR@20 17.671604454517365
-------------------------------------------------------
epoch:  5
[0/6476] Loss: 4.3047
[100/6476] Loss: 3.7599
[200/6476] Loss: 4.1203
[300/6476] Loss: 4.0061
[400/6476] Loss: 4.2138
[500/6476] Loss: 3.7588
[600/6476] Loss: 3.7515
[700/6476] Loss: 3.9898
[800/6476] Loss: 4.0573
[900/6476] Loss: 4.0803
[1000/6476] Loss: 3.8123
[1100/6476] Loss: 4.4258
[1200/6476] Loss: 3.9286
[1300/6476] Loss: 3.8764
[1400/6476] Loss: 4.3117
[1500/6476] Loss: 4.1439
[1600/6476] Loss: 4.2272
[1700/6476] Loss: 4.0428
[1800/6476] Loss: 4.0857
[1900/6476] Loss: 3.8754
[2000/6476] Loss: 4.3681
[2100/6476] Loss: 4.6303
[2200/6476] Loss: 4.2495
[2300/6476] Loss: 3.6984
[2400/6476] Loss: 4.3156
[2500/6476] Loss: 4.4021
[2600/6476] Loss: 4.3280
[2700/6476] Loss: 4.3144
[2800/6476] Loss: 3.9967
[2900/6476] Loss: 4.3159
[3000/6476] Loss: 4.5977
[3100/6476] Loss: 3.9604
[3200/6476] Loss: 4.4991
[3300/6476] Loss: 4.0246
[3400/6476] Loss: 4.1496
[3500/6476] Loss: 4.1351
[3600/6476] Loss: 4.3828
[3700/6476] Loss: 3.8577
[3800/6476] Loss: 4.0991
[3900/6476] Loss: 4.1551
[4000/6476] Loss: 3.8340
[4100/6476] Loss: 4.3578
[4200/6476] Loss: 4.3236
[4300/6476] Loss: 3.7624
[4400/6476] Loss: 3.7615
[4500/6476] Loss: 4.1939
[4600/6476] Loss: 4.0298
[4700/6476] Loss: 4.1174
[4800/6476] Loss: 4.0329
[4900/6476] Loss: 4.6891
[5000/6476] Loss: 4.2944
[5100/6476] Loss: 4.2310
[5200/6476] Loss: 4.0398
[5300/6476] Loss: 3.9838
[5400/6476] Loss: 3.7301
[5500/6476] Loss: 3.8208
[5600/6476] Loss: 3.9707
[5700/6476] Loss: 4.0277
[5800/6476] Loss: 3.8343
[5900/6476] Loss: 4.5943
[6000/6476] Loss: 4.6227
[6100/6476] Loss: 3.9443
[6200/6476] Loss: 4.1274
[6300/6476] Loss: 3.7462
[6400/6476] Loss: 3.9211
	Loss:	26517.164
Recall@20 : 48.68166744709015 , MRR@20 16.970250010490417
-------------------------------------------------------
epoch:  6
[0/6476] Loss: 4.0809
[100/6476] Loss: 3.5517
[200/6476] Loss: 3.8672
[300/6476] Loss: 3.7780
[400/6476] Loss: 4.0148
[500/6476] Loss: 3.5974
[600/6476] Loss: 3.5634
[700/6476] Loss: 3.8127
[800/6476] Loss: 3.8079
[900/6476] Loss: 3.7632
[1000/6476] Loss: 3.5390
[1100/6476] Loss: 4.1453
[1200/6476] Loss: 3.7322
[1300/6476] Loss: 3.5915
[1400/6476] Loss: 4.0913
[1500/6476] Loss: 3.9743
[1600/6476] Loss: 4.0677
[1700/6476] Loss: 3.8054
[1800/6476] Loss: 3.8578
[1900/6476] Loss: 3.6867
[2000/6476] Loss: 4.1134
[2100/6476] Loss: 4.4034
[2200/6476] Loss: 3.9909
[2300/6476] Loss: 3.4314
[2400/6476] Loss: 4.1290
[2500/6476] Loss: 4.0855
[2600/6476] Loss: 4.0710
[2700/6476] Loss: 4.0437
[2800/6476] Loss: 3.7726
[2900/6476] Loss: 3.9925
[3000/6476] Loss: 4.3519
[3100/6476] Loss: 3.6755
[3200/6476] Loss: 4.2962
[3300/6476] Loss: 3.8530
[3400/6476] Loss: 3.8560
[3500/6476] Loss: 3.9602
[3600/6476] Loss: 4.1069
[3700/6476] Loss: 3.5616
[3800/6476] Loss: 3.7925
[3900/6476] Loss: 3.9940
[4000/6476] Loss: 3.6122
[4100/6476] Loss: 4.1309
[4200/6476] Loss: 4.0513
[4300/6476] Loss: 3.5562
[4400/6476] Loss: 3.5502
[4500/6476] Loss: 3.9512
[4600/6476] Loss: 3.7841
[4700/6476] Loss: 3.9263
[4800/6476] Loss: 3.7982
[4900/6476] Loss: 4.4034
[5000/6476] Loss: 4.0538
[5100/6476] Loss: 4.0373
[5200/6476] Loss: 3.8729
[5300/6476] Loss: 3.7667
[5400/6476] Loss: 3.5372
[5500/6476] Loss: 3.6429
[5600/6476] Loss: 3.6708
[5700/6476] Loss: 3.9067
[5800/6476] Loss: 3.6299
[5900/6476] Loss: 4.4071
[6000/6476] Loss: 4.3326
[6100/6476] Loss: 3.6966
[6200/6476] Loss: 3.9107
[6300/6476] Loss: 3.4614
[6400/6476] Loss: 3.6678
	Loss:	24945.543
Recall@20 : 46.78721725940704 , MRR@20 16.294902563095093
```